## <center>Natural Language Processing (Tugas 2)</center>
<strong> NIM  : 1301178418 </strong><br>
<strong> Nama : Bastomy </strong> <br>
<strong> Kelas : ICM-39-GAB </strong>

#### Deskripsi Tugas

Deskripsi
Buatlah program yang mengimplementasikan dua pendekatan POS-tagging. Sebagai input adalah
kalimat Bahasa Indonesia and output adalah kalimat setelah setiap katanya diberi tag yang tepat.
Kedua pendekatan yang harus diimplementasikan adalah:
1. Baseline (menggunakan POS Tag yang paling sering muncul)
2. HMM.

### Import library

In [1]:
import pandas as pd
import re
import operator

<strong>pandas</strong> =  untuk mempermudah membuat array menjadi data tabel <br>
<strong>re</strong> = regular expression untuk membagi string menjadi array berdasarkan tab (\t) <br>
<strong>operator</strong> = untuk mengambil item

### Fungsi untuk membaca data training dan testing menjadi array

In [2]:
def read_data(train):
    dt_train = {'kata':[], 'tag':[]}
    for line in open(train):
        if not line.startswith('<kalimat') and not line.startswith('</kalimat'):
            baris = re.split(r'\t+', line.rstrip('\t'))
            dt_train['kata'].append(baris[0].lower())
            dt_train['tag'].append(baris[1].replace('\n',''))
    data = pd.DataFrame(dt_train)        
    return data

read_data adalah fungsi untuk membaca data .txt menjadi sebuah array <br>
terdapat dua kolom array yaitu kata dan tag  <br>
setiap baris data akan di looping dan di cek jika tidak berawalan <kalimat dan </kalimat maka akan lanjut ke tahap berikutnya yaitu membagi setiap baris menjadi array berdasarkan \t (Tab) <br>
kemudian akan di simpan ke dt_train dan di set menjadi Dataframe untuk di return

### Fungsi untuk menghitung jumlah tag berdasarkan kata

In [3]:
def post_tag_frequency(kata, data_training):
    hitungTag = data_training.loc[data_training['kata'] == kata]['tag'].value_counts()
    if(len(hitungTag) == 0):
        return 'NN','0'
    tagTerbaik = max(hitungTag.items(),key=operator.itemgetter(1))
    return tagTerbaik[0],tagTerbaik[1]

Menghitung jumlah tag berdasarkan kata  <br>
variabel hitungTag akan menyimpan jumlah tag berdasarkan kata  <br>
jika tag bernilai 0 maka akan di terun tag NN dengan jumlah 0  (untuk menghandle kata yg tidak terdaftar)<br>
kemudian akan dicari max dari tag yg tersedia  <br>
fungsi akan mengembalikan Tag dan jumlah tag nya <br>
contoh (PRP   44) <br>

### Fungsi prediksi tag BASELINE

In [4]:
def baseline(data_testing,data_training):
    index = 0 
    tag_correct = 0
    result = {'kata':[], 'prediksi':[], 'tag':[]}
    akurasi = 0
    for i in data_testing['kata']:
        tag,count = post_tag_frequency(i, data_training)
        if(tag == data_testing['tag'][index]):
            tag_correct+=1
        result['kata'].append(i)
        result['prediksi'].append(tag)
        result['tag'].append(data_testing['tag'][index])
        index+=1
    data = pd.DataFrame(result)
    akurasi = (tag_correct/index)*100
    return data,akurasi

Baseline akan memprediksi Tag berdasarkan kemunculan terbanyak <br>
cara kerja fungsi diatas adalah melooping semua kata beserta Tag <br>
kemudian kata tersebut akan diprediksi Tag nya berdasarkan data training <br>
post_tag_frequency akan mengembalikan tag beserta jumlahnya kemunculannya, tag dari post_tag_frequency akan dijadikan prediksi <br>
akurasi dihitung dari jumlah tag yg benar dibagi dengan total seluruh kata

### Read Data

In [5]:
dt_training = read_data('corpus/training.txt')
dt_testing = read_data('corpus/testing.txt')

simpan data training dan testing kedalam variabel

### NO 1. Baseline (menggunakan POS Tag yang paling sering muncul)

In [6]:
data, akurasi = baseline(dt_testing,dt_training)

10 Data testing dan prediksi TAG

In [7]:
data[:10]

,kata,prediksi,tag
0,perusahaan,NN,NN
1,ritel,NN,NN
2,pt,NNP,NNP
3,matahari,NNP,NNP
4,putra,NN,NNP
5,prima,NNP,NNP
6,tbk,NNP,NNP
7,mppa,NN,NNP
8,dalam,IN,IN
9,tiga,CD,CD


Akurasi Tag (persen)

In [8]:
print("akurasi ", akurasi," persen")

akurasi  87.97595190380761  persen


### No 2 HMM

### Import library

In [9]:
from nltk import word_tokenize
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
import time

import library yang akan digunakan seperti :<br>
<strong>word_tokenize</strong> : untuk membuat kalimat menjadi token<br>
<strong>tree</strong> : untuk membuat DecisionTreeClassifier<br>
<strong>DictVectorizer</strong> : untuk merubah feature menjadi matrix vektor<br>
<strong>time</strong> : untuk menghitung lama program di eksekusi<br>

### Fungsi untuk mengekstrak data menjadi kalimat + tag

In [10]:
def read_data_sentences(data):
    sentences = [] ##menyediakan variabel untuk panampungan kalimat
    tags = [] ## variabel untuk manampung tag dari kalimat
    with open(data) as f: 
        content = f.readlines() ## menampung baris setiap data pada variabel content
    contents = [x.strip() for x in content] 
    sentence = [] ## variabel untuk menampung kata dari setiap kalimat
    tag = [] # mvariabel menampung tag dari setiap kata pada suatu kalimat
    for i in contents: 
        if not i.startswith('<kalimat') and not i.startswith('</kalimat'): ##mengecek baris data yang tidak berawalan <kalimat
            content_part = i.split('\t') ## membagi tab menjadi array
            if (len(content_part)==2): 
                sentence.append(content_part[0]) ##menyimpan data kata kedalam array sentence
                tag.append(content_part[1]) ## menyimpan data tag kealam array tag
        elif i.startswith('</kalimat'):
            sentences.append(sentence)
            tags.append(tag)
            sentence = [] ## memasukkan data dari arry kata yang telah membentuk suatu kalimat kedalam array kalimat.
            tag = [] ## memasukkan tag dari kata yang telah membentuk suatu kalimat kedalam array kalimat
    return sentences,tags    

In [29]:
testing_sentences,testing_tags = read_data_sentences('corpus/testing.txt')

testing_sentences[:1]

[['Perusahaan',
  'ritel',
  'PT',
  'Matahari',
  'Putra',
  'Prima',
  'Tbk',
  'MPPA',
  'dalam',
  'tiga',
  'bulan',
  'pertama',
  'yang',
  'berakhir',
  'Maret',
  '2007',
  'berhasil',
  'meraih',
  'laba bersih',
  'Rp',
  '"15,18"',
  'miliar',
  '","',
  'atau',
  'naik',
  '"24,43"',
  'persen',
  'dari',
  'Rp',
  '"12,20"',
  'miliar',
  'pada',
  'periode',
  'sama',
  'tahun',
  'lalu',
  '.']]

<strong>Contoh:</strong> <br>
<table style="border: 1px solid black;">
	<tr style="border: 1px solid black;">
		<td style="border: 1px solid black;" style="border: 1px solid black;">Kalimat</td>
		<td style="border: 1px solid black;">namun</td>
		<td style="border: 1px solid black;">laba</td>
		<td style="border: 1px solid black;">usaha</td>
		<td style="border: 1px solid black;">dalam</td>
		<td style="border: 1px solid black;">periode</td>
		<td style="border: 1px solid black;">tersebut</td>
		<td style="border: 1px solid black;">menyusut</td>
		<td style="border: 1px solid black;">dari</td>
		<td style="border: 1px solid black;">semula</td>
		<td style="border: 1px solid black;">Rp</td>
		<td style="border: 1px solid black;">58,27</td>
		<td style="border: 1px solid black;">miliar</td>
		<td style="border: 1px solid black;">menjadi</td>
		<td style="border: 1px solid black;">Rp</td>
		<td style="border: 1px solid black;">41,39</td>
		<td style="border: 1px solid black;">miliar</td>
		<td style="border: 1px solid black;">.</td>
	</tr>
	<tr style="border: 1px solid black;">
		<td style="border: 1px solid black;" style="border: 1px solid black;">Tag</td>
		<td style="border: 1px solid black;">CC</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">IN</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">PR</td>
		<td style="border: 1px solid black;">VB</td>
		<td style="border: 1px solid black;">IN</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">SYM</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">VB</td>
		<td style="border: 1px solid black;">SYM</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">Z</td>
	</tr>
</table>

In [11]:
def extract_features(sentence, index): ## fungsi untuk ekstraksi fitur
    return {
        'word': sentence[index], ## menampung kata
        'prefix-1': sentence[index][0], ## huruf pertama
        'prefix-2': sentence[index][:2], ## 2 huruf pertama
        'prefix-3': sentence[index][:3], ## 3 huruf pertama
        'suffix-1': sentence[index][-1], ## 1 huruf terakhir
        'suffix-2': sentence[index][-2:], ## 2 huruf terakhir
        'suffix-3': sentence[index][-3:], ## 3 huruf terakhir
        'prev_word': '' if index == 0 else sentence[index - 1], ## kata sebelumnya
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1], ## kata selanjutnya
    }

In [12]:
def transform_to_dataset(sentences, tags):
    X, y = [], [] 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(extract_features(sentences[sentence_idx], index)) ## menampung hasil ekstraksi fitur
            y.append(tags[sentence_idx][index]) ## menampung kata
    return X, y

In [13]:
training_sentences,training_tags = read_data_sentences('corpus/training.txt')
testing_sentences,testing_tags = read_data_sentences('corpus/testing.txt')

In [14]:
X, y = transform_to_dataset(training_sentences, training_tags)

membuat decission tree classifier 

In [15]:
start = time.time()
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y) ## mentraining data kedalam decission tree
finish = time.time()
print('Training completed in ',finish-start,' seconds')

Training completed in  352.99215292930603  seconds


In [16]:
X_test, y_test = transform_to_dataset(testing_sentences, testing_tags) 
##melakukan testing menggunakan model dari hasil decission tree
 
print("Accuracy:")
print(clf.score(X_test, y_test)*100,'persen')

Accuracy:
94.7895791583 persen


In [17]:
# Test model yang sudah dilatih dengan kalimat masukan bebas
from nltk import word_tokenize

def pos_tag(sentence):
    tags = clf.predict([extract_features(sentence, index) for index in range(len(sentence))])
    data = pd.DataFrame({"kata":sentence,'tag':tags})
    return data


In [18]:
result = pos_tag(word_tokenize('Pemerintah Kabupaten Serang memberikan insentif kepada 1.796 Kader Pos Keluarga Berencana dan Kader Subpos KB.'))

Hasil tagging dari penggunaan kalimat bebas

In [19]:
result

,kata,tag
0,Pemerintah,NN
1,Kabupaten,NNP
2,Serang,NNP
3,memberikan,VB
4,insentif,JJ
5,kepada,IN
6,1.796,CD
7,Kader,NNP
8,Pos,NNP
9,Keluarga,NNP
